In [8]:
from gensim.models import Word2Vec
import pandas as pd
import pickle
import time
import logging
import multiprocessing as mp
logging.basicConfig(
	format='%(asctime)s : %(levelname)s : %(message)s',
	level=logging.INFO)

cores = mp.cpu_count()
list_corpus = ['corpus/attraction_tag.list',
               'corpus/hotel_tag.list',
               'corpus/restaurant_tag.list',
               'corpus/eng_attraction_tag.list',
               'corpus/eng_hotel_tag.list',
               'corpus/eng_restaurant_tag.list']

list_tag_model = ['model/attraction_tag.model',
                  'model/hotel_tag.model',
                  'model/restaurant_tag.model']
nodup_model = ['model/attraction_tag_nodup.model',
               'model/hotel_tag_nodup.model',
               'model/restaurant_tag_nodup.model']

params_tag = [{'size':300, 'window':99999, 'min_count':0,        # Attraction
               'workers':cores, 'iter':100, 'sg':1, 'sample':1},
              {'size':300, 'window':99999, 'min_count':0,        # Hotel
               'workers':cores, 'iter':100, 'sg':1, 'sample':1},
              {'size':300, 'window':99999, 'min_count':0,        # Restaurant
               'workers':cores, 'iter':100, 'sg':1, 'sample':1e-3}]

In [9]:
# int to string
for c in list_corpus:
    with open(c, 'rb') as f:
        corpus = pickle.load(f)
    corpus = [[str(pid) for pid in line] for line in corpus]
    with open(c,'wb') as f:
        pickle.dump(corpus, f)

In [10]:
corpora = []
for i in range(3):
    with open(list_corpus[i], 'rb') as f:
        kor = pickle.load(f)
    with open(list_corpus[i+3], 'rb') as f:
        eng = pickle.load(f)
    corpora.append(kor+eng)

In [16]:
#test
corpus = corpora[0]
print('corpus 길이', len(corpus))
total = pd.DataFrame([len(sent) for sent in corpus]).sum()
length = 0
for line in corpus:
    length = max(length, len(line))
print('corpus내 최대 길이', length)
print('corpus내 모든 장소 합', total)

corpus 길이 65862
corpus내 최대 길이 33995
corpus내 모든 장소 합 0    2131509
dtype: int64


In [28]:
spent = []
for i in range(6,7):
    with open(list_corpus[i], 'rb') as f:
        corpus = pickle.load(f)
    start = time.time()
    model = Word2Vec(corpus, **params_tag[0])
    spent.append('Elapsed time: '+str(time.time() - start)+' sec'+' ['+list_tag_model[i]+']')
    model.wv.save(list_tag_model[6])
print(spent)

2018-08-28 16:15:14,728 : INFO : collecting all words and their counts
2018-08-28 16:15:14,729 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-28 16:15:14,771 : INFO : PROGRESS: at sentence #10000, processed 229830 words, keeping 409 word types
2018-08-28 16:15:14,783 : INFO : PROGRESS: at sentence #20000, processed 278251 words, keeping 411 word types
2018-08-28 16:15:14,791 : INFO : PROGRESS: at sentence #30000, processed 300515 words, keeping 412 word types
2018-08-28 16:15:14,800 : INFO : PROGRESS: at sentence #40000, processed 315893 words, keeping 412 word types
2018-08-28 16:15:14,806 : INFO : PROGRESS: at sentence #50000, processed 327591 words, keeping 413 word types
2018-08-28 16:15:14,809 : INFO : collected 417 word types from a corpus of 330320 raw words and 52636 sentences
2018-08-28 16:15:14,809 : INFO : Loading a fresh vocabulary
2018-08-28 16:15:14,811 : INFO : effective_min_count=0 retains 417 unique words (100% of original 417, drops

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U21') dtype('<U21') dtype('<U21')